In [106]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [189]:
df = pd.read_csv('movies.csv', encoding='utf-8', on_bad_lines='skip')

In [190]:
df.shape

(5002, 18)

## Basic Overview of dataset

In [191]:
df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,cast,director,keywords,genre_names
0,FALSE,/yDHYTfA3R0jFYba16jBB1ef8oIt.jpg,"[28, 35, 878]",533535,en,Deadpool & Wolverine,A listless Wade Wilson toils away in civilian ...,5464.692,/8cdWjvZQUExUUTzyp4t6EDMubfO.jpg,24-07-2024,Deadpool & Wolverine,FALSE,7.800,2414.0,"['Ryan Reynolds', 'Hugh Jackman', 'Emma Corrin']",Shawn Levy,"['hero', 'superhero', 'anti hero', 'mutant', '...","['Action', 'Comedy', 'Science Fiction']"
1,FALSE,/stKGOm8UyhuLPR9sZLjs5AkmncA.jpg,"[16, 10751, 12, 35]",1022789,en,Inside Out 2,Teenager Riley's mind headquarters is undergoi...,2641.200,/vpnVM9B6NMmQpWeZvzLvDESb2QY.jpg,11-06-2024,Inside Out 2,FALSE,7.700,3024.0,"['Amy Poehler', 'Maya Hawke', 'Kensington Tall...",Kelsey Mann,"['sadness', 'disgust', 'sequel', 'computer ani...","['Animation', 'Family', 'Adventure', 'Comedy']"
2,FALSE,/4ft6TR9wA6bra0RLL6G7JFDQ5t1.jpg,"[28, 35]",704239,en,The Union,A New Jersey construction worker goes from reg...,1686.712,/d9CTnTHip1RbVi2OQbA2LJJQAGI.jpg,15-08-2024,The Union,FALSE,6.266,396.0,"['Mark Wahlberg', 'Halle Berry', 'J.K. Simmons']",Julian Farino,"['espionage', 'spy', 'old flame', 'miscegenati...","['Action', 'Comedy']"
3,FALSE,/3q01ACG0MWm0DekhvkPFCXyPZSu.jpg,"[28, 80, 53, 35]",573435,en,Bad Boys: Ride or Die,"After their late former Captain is framed, Low...",1751.057,/oGythE98MYleE6mZlGs5oBGkux1.jpg,05-06-2024,Bad Boys: Ride or Die,FALSE,7.580,1719.0,"['Will Smith', 'Martin Lawrence', 'Vanessa Hud...",Adil El Arbi,"['miami, florida', 'sequel', 'on the run', 'po...","['Action', 'Crime', 'Thriller', 'Comedy']"
4,FALSE,/lgkPzcOSnTvjeMnuFzozRO5HHw1.jpg,"[16, 10751, 35, 28]",519182,en,Despicable Me 4,"Gru and Lucy and their girls—Margo, Edith and ...",2051.558,/wWba3TaojhK7NdycRhoQpsG0FaH.jpg,20-06-2024,Despicable Me 4,FALSE,7.300,1257.0,"['Steve Carell', 'Kristen Wiig', 'Will Ferrell']",Chris Renaud,"['superhero', 'villain', 'sequel', 'super vill...","['Animation', 'Family', 'Comedy', 'Action']"


In [110]:
df.sample(5)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,cast,director,keywords,genre_names
2534,FALSE,/77xFxLtD2jGpOrcjJbCez2wZWX1.jpg,[10749],698818,ko,음란 주택: 노예가 된 아내,"Hyun-jung, who moved to a local mansion, was d...",41.757,/1VfuzK2Qx1tIGE1nhn2G2ilF7RN.jpg,06-02-2020,Obscene House: Slave Wife,FALSE,5.500,2.0,"['Sae Bom', 'Kim Min-jeong', 'Min Do-yoon']",Han Dong-Ho,['softcore'],['Romance']
1092,FALSE,/AqzL3Mj6e7qEjS3gHEcccYcQXpq.jpg,[35],1243895,es,Canta y no llores,"A matriarchal family runs a taco food truck, n...",47.276,/8WambNXFVcMyTeCWajNEYfT80Xp.jpg,21-03-2024,Sing and Don't Cry,FALSE,6.976,41.0,"['Michelle Rodríguez', 'Consuelo Duval', 'Patr...",Félix Sabroso,[],['Comedy']
4291,FALSE,/7PZlpdoIOyyFurOyfyBAfYelDFQ.jpg,"[10751, 35]",691683,en,Cheaper by the Dozen,This remake of the beloved classic follows the...,30.686,/qNRsouZh5zmhaE3n4QpLDXzy1gQ.jpg,18-03-2022,Cheaper by the Dozen,FALSE,6.420,243.0,"['Gabrielle Union', 'Zach Braff', 'Erika Chris...",Gail Lerner,"['based on novel or book', 'big family', 'rema...","['Family', 'Comedy']"
4511,FALSE,/kpuah918VrjX5VugqqgecwvbY5d.jpg,"[878, 28, 12]",11867,en,Barb Wire,"A sexy nightclub owner, Barb Wire moonlights a...",29.683,/jW5TLptY7PL1Mllq1g1uMjwXLXT.jpg,02-05-1996,Barb Wire,FALSE,3.955,404.0,"['Pamela Anderson', 'Temuera Morrison', 'Victo...",David Hogan,"['bounty hunter', 'civil war', 'dystopia', 'ba...","['Science Fiction', 'Action', 'Adventure']"
4941,FALSE,/AqLtFRKVX2G0ghMnMnQoSGan4pY.jpg,"[18, 10402]",786,en,Almost Famous,"In 1973, 15-year-old William Miller's unabashe...",37.471,/3rrkyLYbgLj84AYvjhdcJot4JPx.jpg,15-09-2000,Almost Famous,FALSE,7.507,2654.0,"['Billy Crudup', 'Frances McDormand', 'Kate Hu...",Cameron Crowe,"['hotel room', 'concert', ""rock 'n' roll"", 'st...","['Drama', 'Music']"


In [111]:
df.shape

(5002, 18)

In [112]:
df.columns

Index(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'title', 'video', 'vote_average', 'vote_count', 'cast',
       'director', 'keywords', 'genre_names'],
      dtype='object')

## Extracting id and respective poster_path

In [193]:
poster_paths = df[['id', 'poster_path']]

In [195]:
pickle.dump(poster_paths, open('poster_paths.pkl', 'wb'))

## Filtering columns 
Only keep those columns which will help in building recommender system.

In [113]:
df['original_language'].value_counts()

original_language
en         3798
ja          314
ko          153
es          134
fr          121
zh           81
it           68
cn           56
tl           32
ru           29
hi           28
de           25
th           19
sv           17
pt           15
id           14
no           14
pl           12
tr            9
te            7
da            7
ta            6
nl            6
ar            4
kn            3
ca            3
el            3
mn            2
vi            2
sr            2
is            2
fi            2
uk            2
ml            2
bn            2
XXXXXXX       2
cs            1
lt            1
hr            1
ms            1
mk            1
yo            1
Name: count, dtype: int64

In [114]:
df['adult'].value_counts()

adult
FALSE                                                 5000
 Of the domestic exploration for finding my voice.       2
Name: count, dtype: int64

### columns to include
- id
- title
- overview
- genre_names 
- keywords
- cast 
- director

In [115]:
df = df[['id', 'title', 'overview', 'genre_names', 'keywords', 'cast', 'director']]

## Handling duplicate values

In [116]:
df.duplicated().sum()

113

In [117]:
df[df.duplicated()]

,id,title,overview,genre_names,keywords,cast,director
60,299536,Avengers: Infinity War,As the Avengers and their allies have continue...,"['Adventure', 'Action', 'Science Fiction']","['sacrifice', 'magic', 'superhero', 'based on ...","['Robert Downey Jr.', 'Chris Evans', 'Chris He...",Anthony Russo
61,943344,Underground Monster,"Near a remote town, the repeated dumping of to...","['Fantasy', 'Adventure']",[],"['Wei Wei', 'Wei Lu', 'Zhang Chunzhong']",Huang He
62,7451,xXx,Xander Cage is your standard adrenaline junkie...,"['Action', 'Adventure', 'Thriller', 'Crime']","['mission', 'biological weapon', 'sports car',...","['Vin Diesel', 'Asia Argento', 'Marton Csokas']",Rob Cohen
98,1032823,Trap,A father and teen daughter attend a pop concer...,"['Crime', 'Thriller', 'Horror', 'Mystery']","['concert', 'philadelphia, pennsylvania', 'dou...","['Josh Hartnett', 'Ariel Donoghue', 'Saleka']",M. Night Shyamalan
200,866398,The Beekeeper,One man's campaign for vengeance takes on nati...,"['Crime', 'Action', 'Adventure', 'Thriller']","['potus', 'secret government agency', 'fbi age...","['Jason Statham', 'Emmy Raver-Lampman', 'Bobby...",David Ayer
...,...,...,...,...,...,...,...
4824,11284,Freddy's Dead: The Final Nightmare,"Just when you thought it was safe to sleep, Fr...","['Horror', 'Thriller']","['amnesia', 'dream', 'nightmare', 'sadism', 'p...","['Robert Englund', 'Lisa Zane', 'Shon Greenbla...",Rachel Talalay
4862,1072876,Self Reliance,When a man is offered a million dollars to pla...,"['Comedy', 'Thriller']","['boredom', 'homeless', 'dark web', 'reality s...","['Jake Johnson', 'Anna Kendrick', 'Natalie Mor...",Jake Johnson
4922,527776,Overcomer,After reluctantly agreeing to coach cross-coun...,['Drama'],"['sports', 'christianity', 'cross country']","['Alex Kendrick', 'Priscilla C. Shirer', 'Came...",Alex Kendrick
4923,109443,Anchorman 2: The Legend Continues,"With the 70s behind him, San Diego's top rated...",['Comedy'],"['journalism', 'language', 'mustache', 'gang w...","['Will Ferrell', 'Steve Carell', 'Paul Rudd']",Adam McKay


In [118]:
df.drop_duplicates(inplace=True)

## Handling missing values

In [119]:
df.isna().sum()

id              0
title           1
overview       34
genre_names     2
keywords        2
cast            2
director       25
dtype: int64

In [120]:
df[df['title'].isna()]

,id,title,overview,genre_names,keywords,cast,director
4179,457697,NaN,A tarot card reading as a starting point. A d...,NaN,NaN,NaN,NaN


In [121]:
df[df['genre_names'].isna()]

,id,title,overview,genre_names,keywords,cast,director
4179,457697,NaN,A tarot card reading as a starting point. A d...,NaN,NaN,NaN,NaN
4180,01-09-2016,[],6.9,NaN,NaN,NaN,NaN


In [122]:
df[df['keywords'].isna()]

,id,title,overview,genre_names,keywords,cast,director
4179,457697,NaN,A tarot card reading as a starting point. A d...,NaN,NaN,NaN,NaN
4180,01-09-2016,[],6.9,NaN,NaN,NaN,NaN


In [123]:
df.dropna(subset=['title', 'keywords'], inplace=True)

In [124]:
df.isna().sum()

id              0
title           0
overview       34
genre_names     0
keywords        0
cast            0
director       23
dtype: int64

In [125]:
df = df.dropna()

In [126]:
df[df['overview'].isna()]

,id,title,overview,genre_names,keywords,cast,director


## Formatting textual columns into list

In [127]:
df['genre_names'][0]

"['Action', 'Comedy', 'Science Fiction']"

In [128]:
ast.literal_eval("['Action', 'Comedy', 'Science Fiction']")

['Action', 'Comedy', 'Science Fiction']

In [129]:
df['keywords'][0]

"['hero', 'superhero', 'anti hero', 'mutant', 'breaking the fourth wall', 'marvel cinematic universe (mcu)', 'mutants', 'superhero teamup']"

In [130]:
df['cast'][0]

"['Ryan Reynolds', 'Hugh Jackman', 'Emma Corrin']"

In [131]:
df['director'][0]

'Shawn Levy'

In [153]:
def create_list(name):
    return [name]

### Creating lists from all the textual columns

In [133]:
df['genre_names'] = df['genre_names'].apply(ast.literal_eval)
df['keywords'] = df['keywords'].apply(ast.literal_eval)
df['cast'] = df['cast'].apply(ast.literal_eval)
df['director'] = df['director'].apply(create_list)
df['overview'] = df['overview'].apply(lambda x:x.split())

### Removing spaces between words so that meaning remains

In [137]:
def collapse(L):
    l = []
    for i in L:
        l.append(i.replace(' ', ''))
    return l

In [140]:
df['genre_names'].apply(collapse)

0             [Action, Comedy, ScienceFiction]
1       [Animation, Family, Adventure, Comedy]
2                             [Action, Comedy]
3            [Action, Crime, Thriller, Comedy]
4          [Animation, Family, Comedy, Action]
                         ...                  
4997                           [Drama, Comedy]
4998                                [Thriller]
4999             [Action, Drama, History, War]
5000                [Mystery, Thriller, Crime]
5001                                  [Comedy]
Name: genre_names, Length: 4840, dtype: object

In [141]:
df['keywords'].apply(collapse)

0       [hero, superhero, antihero, mutant, breakingth...
1       [sadness, disgust, sequel, computeranimation, ...
2       [espionage, spy, oldflame, miscegenation, cons...
3       [miami,florida, sequel, ontherun, policedetect...
4       [superhero, villain, sequel, supervillain, ill...
                              ...                        
4997    [bondage, lesbianrelationship, torture, basedo...
4998                                     [schoolshooting]
4999    [siblingrelationship, fight, brother, worldwar...
5000    [island, fbi, simulation, training, serialkill...
5001    [mutiny, submarine, u.s.navy, misfit, maverick...
Name: keywords, Length: 4840, dtype: object

In [143]:
df['cast'].apply(collapse)

0           [RyanReynolds, HughJackman, EmmaCorrin]
1        [AmyPoehler, MayaHawke, KensingtonTallman]
2           [MarkWahlberg, HalleBerry, J.K.Simmons]
3       [WillSmith, MartinLawrence, VanessaHudgens]
4           [SteveCarell, KristenWiig, WillFerrell]
                           ...                     
4997    [NorikoKijima, HarunaYoshizumi, YukiMamiya]
4998        [NaomiWatts, ColtonGobbo, SierraMaltby]
4999        [DanielCraig, LievSchreiber, JamieBell]
5000       [KathrynMorris, JonnyLeeMiller, LLCoolJ]
5001     [KelseyGrammer, LaurenHolly, RobSchneider]
Name: cast, Length: 4840, dtype: object

In [150]:
df['overview'] = df['overview'].apply(collapse)
df['genre_names'] = df['genre_names'].apply(collapse)
df['keywords'] = df['keywords'].apply(collapse)
df['cast'] = df['cast'].apply(collapse)
df['director'] = df['director'].apply(collapse)

In [159]:
df['tags'] = df['overview'] + df['genre_names'] + df['keywords'] + df['cast'] + df['director']

### Creating new dataframe by dropping columns which are already included into tags

In [162]:
new_df = df.drop(columns=['overview', 'genre_names', 'keywords', 'cast', 'director'])

In [163]:
new_df

,id,title,tags
0,533535,Deadpool & Wolverine,"[A, listless, Wade, Wilson, toils, away, in, c..."
1,1022789,Inside Out 2,"[Teenager, Riley's, mind, headquarters, is, un..."
2,704239,The Union,"[A, New, Jersey, construction, worker, goes, f..."
3,573435,Bad Boys: Ride or Die,"[After, their, late, former, Captain, is, fram..."
4,519182,Despicable Me 4,"[Gru, and, Lucy, and, their, girls—Margo,, Edi..."
...,...,...,...
4997,297090,The Torture Club,"[The, ""Torture, Club"", is, an, official, after..."
4998,764835,The Desperate Hour,"[A, woman, desperately, races, to, save, her, ..."
4999,13813,Defiance,"[Based, on, a, true, story,, during, World, Wa..."
5000,16617,Mindhunters,"[Trainees, in, the, FBI's, psychological, prof..."


In [165]:
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))

## Vectorizing strings

In [166]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [171]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [172]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [173]:
vector.shape

(4840, 5000)

In [176]:
similarity = cosine_similarity(vector)

In [177]:
similarity

array([[1.        , 0.02550722, 0.07147417, ..., 0.02973505, 0.        ,
        0.02438299],
       [0.02550722, 1.        , 0.07930516, ..., 0.        , 0.        ,
        0.03607265],
       [0.07147417, 0.07930516, 1.        , ..., 0.03081668, 0.        ,
        0.02526993],
       ...,
       [0.02973505, 0.        , 0.03081668, ..., 1.        , 0.03268602,
        0.04205171],
       [0.        , 0.        , 0.        , ..., 0.03268602, 1.        ,
        0.        ],
       [0.02438299, 0.03607265, 0.02526993, ..., 0.04205171, 0.        ,
        1.        ]])

In [180]:
new_df[new_df['title'] == 'The Lego Movie'].index[0]

1988

In [181]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x:x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [185]:
recommend('Deadpool & Wolverine')

Deadpool 2
Deadpool
Superhero Movie
Deadpool: No Good Deed
Ultraman: Rising


In [187]:
import pickle

In [188]:
pickle.dump(new_df, open('movies.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))

### Processing for fetching poster path

In [202]:
poster_paths[poster_paths['id'] == '704239']['poster_path'].values[0]

'/d9CTnTHip1RbVi2OQbA2LJJQAGI.jpg'

In [203]:
new_df[new_df['id'] == '704239']

,id,title,tags
2,704239,The Union,A New Jersey construction worker goes from reg...
